# Construction du modèle de reconnaissance de caractère basé sur le dataset MIST

Dans ce notebook sera présenté la création du modèle qui permettra de **classifier** les lettres de l'alphabet de a-z et A-Z.

Pour ce faire nous allons utiliser le dataset d'images prétraités créés par le notebook [NIST-preprocessing](/notebooks/notebooks/character_recognition/NIST-preprocessing.ipynb)

Pour exécuter ce notebook, veillez à ce que le jeu de données soit bien sous la forme suivante.
- data
    - processed
         - NIST-dataset
            - train
                - a000001.png
                - a000002.png
                ...
                ...
                - a00000n.png
            - test_set
                - a000001.png
                - a000002.png
                ...
                ...
                - a00000n.png


## Importation des dépendances

In [8]:
import os
import datetime
import multiprocessing as mp
from string import ascii_lowercase, ascii_uppercase

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-03-07 18:16:53.173966: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Sélection des devices

In [9]:
from tensorflow.python.client import device_lib

print("Num of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_lib.list_local_devices()

Num of GPUs Available:  1


2023-03-07 18:16:55.686784: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-07 18:16:55.723400: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-07 18:16:55.723601: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 609870800427251283
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6484983808
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6504015122808818016
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

Num of GPUs Available:  1


2023-03-07 18:16:56.770837: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-07 18:16:56.771137: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-07 18:16:56.771368: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14004107442866400038
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6484983808
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 2338978603911492168
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060 SUPER, pci bus id: 0000:26:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [10]:
tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2023-03-07 18:16:56.729823: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


True

2023-03-07 18:16:56.730025: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-07 18:16:56.730169: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-07 18:16:56.730353: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

True

## Initialisation de tensorboard

In [11]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Définition des paramètres du notebook

In [12]:
NIST_PROCESSED_PATH = "../../data/processed/NIST-dataset"
TRAIN_SET_PATH = os.path.join(NIST_PROCESSED_PATH, "train", "nist_processed_train.csv")
TEST_SET_PATH = os.path.join(NIST_PROCESSED_PATH, "test", "nist_processed_test.csv")
IMG_SIZE = 41
CHUNK_SIZE = 25000
PANDA_CSV_ENGINE = "c"

## Importation du dataset

Dans cette section, nous allons charger le dataset afin d'obtenir un train_data_set, train_label_set, ainsi qu'un test_data_set et un test_label_set

### Chargement du dataset depuis le disque

In [13]:
def get_y_original_shape(y_raw_set):
    x  = y_raw_set.shape
    return y_raw_set.reshape(x[0], 1)

def get_x_original_shape(x_raw_set):
    x, y = x_raw_set.shape
    return x_raw_set.reshape(x, IMG_SIZE, IMG_SIZE)

In [14]:
def get_y_and_x_set_from_dataframe(dataframe):
    y_raw_set = dataframe["label"].to_numpy(dtype=str)
    x_raw_set = dataframe.iloc[:, 2:].to_numpy(dtype=np.uint8)

    y_set = get_y_original_shape(y_raw_set)
    x_set = get_x_original_shape(x_raw_set)

    return y_set, x_set

In [15]:
def get_x_y_from_csv_multi_process(path, engine, chunk_size):
    chunks = pd.read_csv(path, chunksize=chunk_size, engine=engine)
    cpu_count = mp.cpu_count()

    with mp.Pool(cpu_count) as pool:
        data_list = [data for data in pool.map(get_y_and_x_set_from_dataframe, chunks)]

    data = list(zip(*data_list))
    y_set = np.concatenate(data[0])
    x_set = np.concatenate(data[1])

    return y_set, x_set


In [16]:
y_train, x_train = get_x_y_from_csv_multi_process(TRAIN_SET_PATH, PANDA_CSV_ENGINE, CHUNK_SIZE)
y_test, x_test = get_x_y_from_csv_multi_process(TEST_SET_PATH, PANDA_CSV_ENGINE, CHUNK_SIZE)

print(y_train.shape)
print(x_train.shape)
print(y_test.shape)
print(x_test.shape)

(287911, 1)
(287911, 41, 41)
(287911, 1)
(287911, 41, 41)


## Encodage des labels

On récupère tous les labels possibles

In [17]:
label_list = [char for char in ascii_lowercase + ascii_uppercase]
print(label_list[0:5])

['a', 'b', 'c', 'd', 'e']


On définit l'encoder à utiliser

In [18]:
encoder = LabelBinarizer()
encoder.fit(label_list)

LabelBinarizer()

On encode nos données au format OneShot

In [19]:
y_encoded_train = encoder.transform(y_train)
y_encoded_test = encoder.transform(y_test)
y_encoded_train.shape

(287911, 52)

## Création du modèle de classification

In [20]:
model = keras.Sequential([
    keras.Input(shape=(41,41)),
    layers.Flatten(input_shape=(41, 41)),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(52, activation="softmax"),
])

model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["acc", "categorical_crossentropy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 1681)              0         
                                                                 
 dense (Dense)               (None, 100)               168200    
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 52)                5252      
                                                                 
Total params: 203,752
Trainable params: 203,752
Non-trai

2023-03-07 18:18:18.428531: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-07 18:18:18.429322: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-03-07 18:18:18.429470: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

## Configuration de tensorboard si installé

In [21]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

## Entrainement du modèle

In [ ]:
model.fit(x=x_train, y=y_encoded_train, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20


2023-03-07 18:18:23.235740: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f79fc00c160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-07 18:18:23.235769: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 2060 SUPER, Compute Capability 7.5
2023-03-07 18:18:23.453038: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-03-07 18:18:23.748848: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8998/8998 [==============================] - 30s 3ms/step - loss: 1.7272 - acc: 0.5306 - categorical_crossentropy: 1.7272
Epoch 2/20
8998/8998 [==============================] - 28s 3ms/step - loss: 0.9812 - acc: 0.7070 - categorical_crossentropy: 0.9812
Epoch 3/20
8998/8998 [==============================] - 26s 3ms/step - loss: 0.8236 - acc: 0.7464 - categorical_crossentropy: 0.8236
Epoch 4/20
8998/8998 [==============================] - 26s 3ms/step - loss: 0.7394 - acc: 0.7689 - categorical_crossentropy: 0.7394
Epoch 5/20
8998/8998 [==============================] - 27s 3ms/step - loss: 0.6879 - acc: 0.7815 - categorical_crossentropy: 0.6879
Epoch 6/20
8998/8998 [==============================] - 27s 3ms/step - loss: 0.6510 - acc: 0.7913 - categorical_crossentropy: 0.6510
Epoch 7/20
8998/8998 [==============================] - 27s 3ms/step - loss: 0.6206 - acc: 0.7984 - categorical_crossentropy: 0.6206
Epoch 8/20
8998/8998 [==============================] - 27s 3ms/step - loss: 0.5

## Test du modèle sur le jeu de test

In [ ]:
model.evaluate(x_test, y_encoded_test, verbose=2)

In [ ]:
%tensorboard --logdir {log_dir}